In [5]:
#import requests
#from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import time

import seaborn
from matplotlib import pyplot

from sklearn.linear_model import LogisticRegression

%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

ModuleNotFoundError: No module named 'pandas'

In [ ]:
chess_games = pd.read_csv('chess_games.csv')

In [ ]:
#Data Cleaning - convert date and time columns to datatime objects
chess_games['UTCDate'] = pd.to_datetime(chess_games['UTCDate'])
chess_games['UTCTime'] = pd.to_datetime(chess_games['UTCTime'])

#display(chess_games)
#chess_games['Event'] = chess_games['Event'].astype(str)

#chess_games['ECO'] = chess_games['ECO'].astype('|S')
display(chess_games.head())
print(chess_games.dtypes)

In [ ]:
#checking for unusual values
#%sql SELECT * FROM chess_games WHERE Result = '*'
#%sql SELECT DISTINCT Termination FROM chess_games
#%sql SELECT * FROM chess_games WHERE Termination = 'Rules infraction'

#filter out Termination = Abandoned, Rules infraction, filtering out TimeControl = '-'
#checking that there are no games where result is * (meaning incomplete)
completed_chess_games = %sql SELECT * FROM chess_games \
WHERE Termination NOT IN ('Abandoned', 'Rules Infraction') \
    AND TimeControl != '-' \
    AND Result != '*'


#display(completed_chess_games.head())



#separating TimeControl into 2 columns (base & additional)

In [ ]:
#filter out all NaN values
completed_chess_games = completed_chess_games.dropna()
completed_chess_games.head()

In [ ]:
#separating TimeControl into 2 columns (base & additional)
TOP_completed_chess_games = completed_chess_games.head(10)

In [ ]:
#get the time control split into two columns - Base and Incremental
time_control_df = TOP_completed_chess_games.TimeControl.apply(lambda x: pd.Series(str(x).split("+")))

time_control_df = time_control_df.rename(columns={0:"Base", 1:"Increment"})
time_control_df

In [ ]:
#Join old and new tables together
time_control_split_df = pd.concat([TOP_completed_chess_games, time_control_df], axis=1)
display(time_control_split_df.head())

In [ ]:

#%sql SELECT * FROM chess_games WHERE Result = '*'
%sql SELECT DISTINCT Termination FROM chess_games
#%sql SELECT * FROM chess_games WHERE Termination = 'Rules infraction'
#filter out Termination = Abandoned, Rules infraction, filtering out TimeControl = '-'

#filter out all NaN values

#separating TimeControl into 2 columns (base & additional)

In [ ]:
#Summary Statistics and Descriptive Statistics
#%sql top_ten_openings << SELECT Opening, COUNT(White) AS Games_Won FROM chess_games WHERE Result = '1-0' GROUP BY White, Opening ORDER BY Games_Won DESC LIMIT 10

